<a href="https://colab.research.google.com/github/kirmanioussema12/NLP/blob/main/Analyse_s%C3%A9mantique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # Pour les synonymes dans plusieurs langues


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
from nltk.corpus import wordnet as wn
car_synset_1 = wn.synset('car.n.01')
car_synset_2 = wn.synset('car.n.02')
automobile_synset = wn.synset('automobile.n.01')

In [3]:
similarity_1 = car_synset_1.path_similarity(automobile_synset)
similarity_2 = car_synset_2.path_similarity(automobile_synset)

In [4]:
print(f"Similarité entre car.n.01 et automobile.n.01: {similarity_1}")
print(f"Similarité entre car.n.02 et automobile.n.01: {similarity_2}")

Similarité entre car.n.01 et automobile.n.01: 1.0
Similarité entre car.n.02 et automobile.n.01: 0.2


In [5]:
hyponyms_car_1 = car_synset_1.hyponyms()
hyponyms_lemmas = [lemma.name() for synset in hyponyms_car_1 for lemma in synset.lemmas()]



In [6]:
print("Lemmes des hyponymes de car.n.01:")
for lemma in hyponyms_lemmas:
    print(lemma)

Lemmes des hyponymes de car.n.01:
ambulance
beach_wagon
station_wagon
wagon
estate_car
beach_waggon
station_waggon
waggon
bus
jalopy
heap
cab
hack
taxi
taxicab
compact
compact_car
convertible
coupe
cruiser
police_cruiser
patrol_car
police_car
prowl_car
squad_car
electric
electric_automobile
electric_car
gas_guzzler
hardtop
hatchback
horseless_carriage
hot_rod
hot-rod
jeep
landrover
limousine
limo
loaner
minicar
minivan
Model_T
pace_car
racer
race_car
racing_car
roadster
runabout
two-seater
sedan
saloon
sport_utility
sport_utility_vehicle
S.U.V.
SUV
sports_car
sport_car
Stanley_Steamer
stock_car
subcompact
subcompact_car
touring_car
phaeton
tourer
used-car
secondhand_car
